In [17]:
#testing multiprocessing

In [1]:
import apache_beam

In [2]:
import os
import weatherbench2
import xarray as xr
import math
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.evaluation import evaluate_in_memory
from weatherbench2 import config
import numpy as np
import sigkernel
import torch
from einops import rearrange
from itertools import product
import cython
import matplotlib.pyplot  as plt
import tqdm
import Functions as fu
import line_profiler

In [3]:
#Observations
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'
observations = xr.open_zarr(obs_path)

forecast2 = xr.open_zarr('gs://weatherbench2/datasets/ifs_ens/2018-2022-64x32_equiangular_conservative_mean.zarr')
forecast2

forecast_path = 'gs://weatherbench2/datasets/ifs_ens/2018-2022-64x32_equiangular_conservative.zarr'
forecast1 = xr.open_zarr(forecast_path)
forecast1

<xarray.Dataset>
Dimensions:                   (time: 3652, number: 50,
                               prediction_timedelta: 61, longitude: 64,
                               latitude: 32, level: 3)
Coordinates:
  * latitude                  (latitude) float64 -87.19 -81.56 ... 81.56 87.19
  * level                     (level) int32 500 700 850
  * longitude                 (longitude) float64 0.0 5.625 ... 348.8 354.4
  * number                    (number) int32 1 2 3 4 5 6 7 ... 45 46 47 48 49 50
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 00:00:00...
  * time                      (time) datetime64[ns] 2018-01-01 ... 2022-12-31...
Data variables: (12/15)
    10m_u_component_of_wind   (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind   (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    10m_wind_speed            (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    2m_temperature            (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    geopotential              (time, number, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 3, 64, 32), meta=np.ndarray>
    mean_sea_level_pressure   (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    ...                        ...
    total_precipitation       (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    total_precipitation_24hr  (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    total_precipitation_6hr   (time, number, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 64, 32), meta=np.ndarray>
    u_component_of_wind       (time, number, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 3, 64, 32), meta=np.ndarray>
    v_component_of_wind       (time, number, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 3, 64, 32), meta=np.ndarray>
    wind_speed                (time, number, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 50, 1, 3, 64, 32), meta=np.ndarray>

In [4]:
score = fu.ensworkflow(observations, forecast1, 1,61,0)

12
[range(0, 3), range(3, 6), range(6, 9), range(9, 12), range(12, 15), range(15, 18), range(18, 21), range(21, 24), range(24, 27), range(27, 30), range(30, 32)]
start process


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [31]:
def pkparallel_lat_splitens(lat_chunk, observations_chunk, forecasts_chunk, zero):
    """
    Function to compute results for a chunk of latitudes.
    """
    static_kernel = sigkernel.Linear_ID_Kernel()
    dyadic_order = 2
    signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

    time = forecasts_chunk.shape[0]
    number = forecasts_chunk.shape[1]
    lead = forecasts_chunk.shape[2]
    latlength = forecasts_chunk.shape[4]

    pkarray = np.zeros((latlength, time, lead)) ##

    for lat in range(latlength):
        print(lat)
        for t in range(time): ### Change with batching
            #print(t, "time")
            for lag in range(2, lead + 1):
                #print(lag, "lag")
                #print(forecasts_chunk[t,:,0:lag,:,lat].shape)
                X = torch.tensor(sigkernel.transform(forecasts_chunk[t,:,0:lag,:,lat], scale = 1, at = True, ll = False), dtype=torch.double)
                #print(X.shape)
                y = torch.tensor(sigkernel.transform(np.expand_dims(observations_chunk[2*t+zero:2*t+zero+lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)

                score = signature_kernel.compute_scoring_rule(X,y)

                pkarray[lat,t,lag-1] = score

    return pkarray

In [29]:
def ensworkflowtest(observations, forecasts, days, lag,zero):
    ob, fo = fu.timecutting(observations,forecasts,days,lag+zero)
    ob = ob.values
    fo = fo[:,0:2,0:lag,:,:].values #0 to 2
    ob, fo = fu.scalebyobsadjusted(ob,fo,fo.shape[3])
    pkparallel_lat_splitens("yo",ob,fo,zero)



In [ ]:
ensworkflowtest(observations, forecast1, 1,61,0) #0-2 computational time 52 seconds, but 0 to 50 is 30mins to 1 hour

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [12]:
newpk, pkarraylat, distance,score = fu.workflowfulladjusted(observations, forecast2, 2,61,0)

32
0
(1, 2, 65)
torch.Size([1, 2, 65])
(1, 3, 65)
torch.Size([1, 3, 65])
(1, 4, 65)
torch.Size([1, 4, 65])
(1, 5, 65)
torch.Size([1, 5, 65])
(1, 6, 65)
torch.Size([1, 6, 65])
(1, 7, 65)
torch.Size([1, 7, 65])
(1, 8, 65)
torch.Size([1, 8, 65])
(1, 9, 65)
torch.Size([1, 9, 65])
(1, 10, 65)
torch.Size([1, 10, 65])
(1, 11, 65)
torch.Size([1, 11, 65])
(1, 12, 65)
torch.Size([1, 12, 65])
(1, 13, 65)
torch.Size([1, 13, 65])
(1, 14, 65)
torch.Size([1, 14, 65])
(1, 15, 65)
torch.Size([1, 15, 65])
(1, 16, 65)
torch.Size([1, 16, 65])
(1, 17, 65)
torch.Size([1, 17, 65])
(1, 18, 65)
torch.Size([1, 18, 65])
(1, 19, 65)
torch.Size([1, 19, 65])
(1, 20, 65)
torch.Size([1, 20, 65])
(1, 21, 65)
torch.Size([1, 21, 65])
(1, 22, 65)
torch.Size([1, 22, 65])
(1, 23, 65)
torch.Size([1, 23, 65])
(1, 24, 65)
torch.Size([1, 24, 65])
(1, 25, 65)
torch.Size([1, 25, 65])
(1, 26, 65)
torch.Size([1, 26, 65])
(1, 27, 65)
torch.Size([1, 27, 65])
(1, 28, 65)
torch.Size([1, 28, 65])
(1, 29, 65)
torch.Size([1, 29, 65])
(1,

In [13]:
distance

array([0.00000000e+00, 3.02104863e-03, 5.85099162e-03, 7.31116702e-03,
       1.18141360e-02, 1.42991192e-02, 2.19586507e-02, 2.94111369e-02,
       4.15630754e-02, 5.44458802e-02, 7.48854381e-02, 9.43882855e-02,
       1.22598102e-01, 1.52838572e-01, 1.94489235e-01, 2.34742682e-01,
       2.90370724e-01, 3.46608539e-01, 4.19483785e-01, 4.86859925e-01,
       5.61840669e-01, 6.30331833e-01, 7.12479410e-01, 8.00033017e-01,
       9.10924494e-01, 1.01599473e+00, 1.13486299e+00, 1.24154637e+00,
       1.36069708e+00, 1.46778916e+00, 1.59119510e+00, 1.70352131e+00,
       1.83860080e+00, 1.96835005e+00, 2.11068919e+00, 2.26477797e+00,
       2.42923553e+00, 2.59853224e+00, 2.76589286e+00, 2.91595043e+00,
       3.06740399e+00, 3.19580758e+00, 3.34796602e+00, 3.48096795e+00,
       3.64703562e+00, 3.78142711e+00, 3.93717525e+00, 4.05184217e+00,
       4.18321646e+00, 4.29440599e+00, 4.43753116e+00, 4.58499644e+00,
       4.76582380e+00, 4.94503527e+00, 5.15583450e+00, 5.35903562e+00,
      

In [14]:
newpk1, pkarraylat1, distance1,score1 = fu.workflowfullparallel(observations, forecast2, 2,61,0)

12
[range(0, 3), range(3, 6), range(6, 9), range(9, 12), range(12, 15), range(15, 18), range(18, 21), range(21, 24), range(24, 27), range(27, 30), range(30, 32)]
32


In [15]:
distance1

array([0.00000000e+00, 3.02104863e-03, 5.85099162e-03, 7.31116702e-03,
       1.18141360e-02, 1.42991192e-02, 2.19586507e-02, 2.94111369e-02,
       4.15630754e-02, 5.44458802e-02, 7.48854381e-02, 9.43882855e-02,
       1.22598102e-01, 1.52838572e-01, 1.94489235e-01, 2.34742682e-01,
       2.90370724e-01, 3.46608539e-01, 4.19483785e-01, 4.86859925e-01,
       5.61840669e-01, 6.30331833e-01, 7.12479410e-01, 8.00033017e-01,
       9.10924494e-01, 1.01599473e+00, 1.13486299e+00, 1.24154637e+00,
       1.36069708e+00, 1.46778916e+00, 1.59119510e+00, 1.70352131e+00,
       1.83860080e+00, 1.96835005e+00, 2.11068919e+00, 2.26477797e+00,
       2.42923553e+00, 2.59853224e+00, 2.76589286e+00, 2.91595043e+00,
       3.06740399e+00, 3.19580758e+00, 3.34796602e+00, 3.48096795e+00,
       3.64703562e+00, 3.78142711e+00, 3.93717525e+00, 4.05184217e+00,
       4.18321646e+00, 4.29440599e+00, 4.43753116e+00, 4.58499644e+00,
       4.76582380e+00, 4.94503527e+00, 5.15583450e+00, 5.35903562e+00,
      

In [16]:
#2x speedup on my own 6 core pc


In [3]:
obs_pathlarge = 'gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr'
observations240121 = xr.open_zarr(obs_pathlarge)
observations240121

<xarray.Dataset>
Dimensions:                                           (time: 93544,
                                                       longitude: 240,
                                                       latitude: 121, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -90....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    above_ground                                      (time, level, longitude, latitude) float32 dask.array<chunksize=(8, 13, 240, 121), meta=np.ndarray>
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    vorticity                                         (time, level, longitude, latitude) float32 dask.array<chunksize=(8, 13, 240, 121), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 dask.array<chunksize=(8, 13, 240, 121), meta=np.ndarray>

In [4]:
forecast2larger = xr.open_zarr('gs://weatherbench2/datasets/ifs_ens/2018-2022-240x121_equiangular_with_poles_conservative_mean.zarr')
forecast2larger 

<xarray.Dataset>
Dimensions:                   (time: 3652, prediction_timedelta: 61,
                               longitude: 240, latitude: 121, level: 3)
Coordinates:
  * latitude                  (latitude) float64 -90.0 -88.5 -87.0 ... 88.5 90.0
  * level                     (level) int32 500 700 850
  * longitude                 (longitude) float64 0.0 1.5 3.0 ... 357.0 358.5
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 00:00:00...
  * time                      (time) datetime64[ns] 2018-01-01 ... 2022-12-31...
Data variables: (12/15)
    10m_u_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    10m_v_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    10m_wind_speed            (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    2m_temperature            (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    geopotential              (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>
    mean_sea_level_pressure   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    ...                        ...
    total_precipitation       (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    total_precipitation_24hr  (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    total_precipitation_6hr   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    u_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>
    v_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>
    wind_speed                (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>

In [6]:
newpk24021, pkarraylat24021, distance24021,score24021 = fu.workflowfullparallel(observations240121, forecast2larger, days = 3, lag = 61, zero = 0)
#No batching, gram
#Cuda 1.09
#No cuda 1.02

#No batching, kernel
# 1.05 
#No cuda

#Batching, kernel
#Cuda
#No Cuda 0.53

#Compared to basic 1 day 2.34 

#3 Day vs batching,kernel 6.27, 1.45
# #Almost 4x speedup, getting better as t increases 

# Day = 1, 3x Speedup
# Day = 3, 4x Speedup etc..


12
[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]
121


In [5]:
newpk2402, pkarraylat2402, distance2402,score2402 = fu.workflowfulladjusted(observations240121, forecast2larger, days = 3, lag = 61, zero = 0)

121
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [11]:
newpk24021, pkarraylat24021, distance24021,score24021 = fu.workflowfullparallel(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)

[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]
121


In [12]:
distance2402

array([0.00000000e+00, 1.71464355e-02, 3.17276961e-02, 4.01302893e-02,
       6.20408944e-02, 7.50415059e-02, 1.08573007e-01, 1.42175321e-01,
       1.99310197e-01, 2.58910818e-01, 3.51433291e-01, 4.33328455e-01,
       5.56378924e-01, 6.86123318e-01, 8.70540926e-01, 1.06759478e+00,
       1.32919964e+00, 1.60316356e+00, 1.93152378e+00, 2.24503733e+00,
       2.54287802e+00, 2.77633475e+00, 3.05719106e+00, 3.32395365e+00,
       3.67459969e+00, 4.03696783e+00, 4.49215834e+00, 4.97283179e+00,
       5.55609454e+00, 6.09129251e+00, 6.68827905e+00, 7.19613735e+00,
       7.78204220e+00, 8.28403849e+00, 8.83122077e+00, 9.35093443e+00,
       9.93636977e+00, 1.05398588e+01, 1.11173418e+01, 1.17173323e+01,
       1.23193982e+01, 1.29464400e+01, 1.36383757e+01, 1.42829647e+01,
       1.50538949e+01, 1.56138613e+01, 1.63235313e+01, 1.67319218e+01,
       1.72128375e+01, 1.74977932e+01, 1.78796046e+01, 1.82812652e+01,
       1.88307677e+01, 1.94611275e+01, 2.02805066e+01, 2.10914804e+01,
      

In [13]:
distance24021

array([0.00000000e+00, 1.71464355e-02, 3.17276961e-02, 4.01302893e-02,
       6.20408944e-02, 7.50415059e-02, 1.08573007e-01, 1.42175321e-01,
       1.99310197e-01, 2.58910818e-01, 3.51433291e-01, 4.33328455e-01,
       5.56378924e-01, 6.86123318e-01, 8.70540926e-01, 1.06759478e+00,
       1.32919964e+00, 1.60316356e+00, 1.93152378e+00, 2.24503733e+00,
       2.54287802e+00, 2.77633475e+00, 3.05719106e+00, 3.32395365e+00,
       3.67459969e+00, 4.03696783e+00, 4.49215834e+00, 4.97283179e+00,
       5.55609454e+00, 6.09129251e+00, 6.68827905e+00, 7.19613735e+00,
       7.78204220e+00, 8.28403849e+00, 8.83122077e+00, 9.35093443e+00,
       9.93636977e+00, 1.05398588e+01, 1.11173418e+01, 1.17173323e+01,
       1.23193982e+01, 1.29464400e+01, 1.36383757e+01, 1.42829647e+01,
       1.50538949e+01, 1.56138613e+01, 1.63235313e+01, 1.67319218e+01,
       1.72128375e+01, 1.74977932e+01, 1.78796046e+01, 1.82812652e+01,
       1.88307677e+01, 1.94611275e+01, 2.02805066e+01, 2.10914804e+01,
      

In [14]:
newpk24024, pkarraylat24024, distance24024,score24024 = fu.workflowfulladjusted(observations240121, forecast2larger, days = 4, lag = 61, zero = 0)

121
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [15]:
newpk240241, pkarraylat240241, distance240241,score240241 = fu.workflowfullparallel(observations240121, forecast2larger, days = 4, lag = 61, zero = 0)

[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]
121


In [17]:
import cProfile
import pstats

# Profile the pkparallel function
with cProfile.Profile() as pr:
    newpk24021, pkarraylat24021, distance24021,score24021 = fu.workflowfullparallel(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats(20)

[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]


KeyboardInterrupt: 

In [5]:
#Same 2.2/2.3 speedup
newpk24021, pkarraylat24021, distance24021,score24021 = fu.workflowfullparallel(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)

12
[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]
121


In [21]:
%load_ext line_profiler

In [24]:
%lprun -f fu.workflowfullparallel -f fu.pkparallel -f fu.pkparallel_lat_split fu.workflowfullparallel(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)


[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]
121


Timer unit: 1e-07 s

Total time: 67.7277 s
File: c:\Users\arche\4th Year Diss\Archer-4th-Year-Diss\Term2\Functions.py
Function: workflowfullparallel at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def workflowfullparallel(observations, forecasts, days, lag,zero):
    13         1      82546.0  82546.0      0.0      ob, fo = timecutting(observations,forecasts,days,lag+zero)
    14         1    9982936.0    1e+07      1.5      ob = ob.values
    15         1    9608580.0    1e+07      1.4      fo = fo[:,0:lag,:,:].values
    16         1     252426.0 252426.0      0.0      ob, fo = scalebyobsadjusted(ob,fo,fo.shape[2])
    17         1  657350258.0    7e+08     97.1      pkarray, pkarraylat, distance, score = pkparallel(ob,fo,zero)
    18                                           
    19         1         17.0     17.0      0.0      return (pkarray, pkarraylat, distance, score)

Total time: 0 s
File: c:\Users\arc

In [27]:
%lprun -f fu.workflowfulladjusted -f fu.pkfulladjusted -f sigkernel.SigKernel.compute_Gram fu.workflowfulladjusted(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)


121
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


Timer unit: 1e-07 s

Total time: 177.546 s
File: c:\Users\arche\4th Year Diss\Archer-4th-Year-Diss\Term2\Functions.py
Function: workflowfulladjusted at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def workflowfulladjusted(observations, forecasts, days, lag,zero):
    23         1      71695.0  71695.0      0.0      ob, fo = timecutting(observations,forecasts,days,lag+zero)
    24         1    9906426.0    1e+07      0.6      ob = ob.values
    25         1   11371078.0    1e+07      0.6      fo = fo[:,0:lag,:,:].values
    26         1     237947.0 237947.0      0.0      ob, fo = scalebyobsadjusted(ob,fo,fo.shape[2])
    27         1 1753875478.0    2e+09     98.8      newpk, pkarraylat, distance,score = pkfulladjusted(ob,fo,zero)
    28                                           
    29         1         15.0     15.0      0.0      return (newpk, pkarraylat, distance,score)

Total time: 175.051 s
File: c:\Users

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')